In [ ]:
# PARAMETERS (for papermill)
# Đường dẫn tuyệt đối tới file jar đã tải
SPMF_JAR_PATH = r"D:\DataEngineer\DataMining\shopping_cart_advanced_analysis\src\spmf.jar" 

# Đường dẫn dữ liệu chuẩn bị từ Bước 2
INPUT_SPMF = "data/processed/utility_db.txt"

# Đường dẫn lưu kết quả thô từ thuật toán EFIM
OUTPUT_RAW_HUI = "data/processed/raw_hui.txt"

# Ngưỡng doanh thu (Utility) tối thiểu để lọc tập mục giá trị cao
MIN_UTIL = 1000  

# Đường dẫn lưu kết quả cuối cùng dạng CSV
OUTPUT_CSV_RESULT = "data/processed/high_utility_results.csv"

In [ ]:
import subprocess
import os
import time

def run_huim_mining():
    if not os.path.exists(SPMF_JAR_PATH):
        print(f"LỖI: Không tìm thấy file SPMF tại {SPMF_JAR_PATH}")
        return

    # Lệnh thực thi trực tiếp qua Java: java -jar spmf.jar run EFIM input output min_util
    command = [
        "java", "-jar", SPMF_JAR_PATH, 
        "run", "EFIM", 
        INPUT_SPMF, 
        OUTPUT_RAW_HUI, 
        str(MIN_UTIL)
    ]
    
    print(f"--- Đang bắt đầu khai phá High-Utility Itemsets (Thuật toán EFIM) ---")
    print(f"--- Ngưỡng tối thiểu: {MIN_UTIL} ---")
    
    start_time = time.time()
    # Chạy lệnh hệ thống và bắt lỗi nếu có
    process = subprocess.run(command, capture_output=True, text=True)
    end_time = time.time()
    
    if process.returncode == 0:
        print(f"Thành công! Thời gian chạy: {end_time - start_time:.2f} giây")
    else:
        print("Lỗi khi thực thi thuật toán:")
        print(process.stderr)

run_huim_mining()

In [ ]:
import pandas as pd
import pickle

# Load item mapping
with open("data/processed/item_mapping.pkl", "rb") as f:
    mapping = pickle.load(f)
id_to_item = mapping["id_to_item"]

results = []
if os.path.exists(OUTPUT_RAW_HUI):
    with open(OUTPUT_RAW_HUI, "r") as f:
        for line in f:
            if "#UTIL:" in line:
                # Định dạng SPMF: Item1 Item2 #UTIL: 5000
                parts = line.strip().split(" #UTIL: ")
                itemset_ids = parts[0].strip().split()
                itemset_codes = " ".join(id_to_item[int(id)] for id in itemset_ids)
                results.append({
                    "Itemset": itemset_codes,
                    "Total_Utility": float(parts[1].strip())
                })

# Chuyển thành DataFrame để dễ xử lý và sắp xếp
if results:
    df_hui = pd.DataFrame(results).sort_values("Total_Utility", ascending=False)

    # Lưu kết quả ra CSV để dùng cho báo cáo
    df_hui.to_csv(OUTPUT_CSV_RESULT, index=False)

    print(f"Đã tìm thấy {len(df_hui)} tập mục có giá trị cao vượt ngưỡng {MIN_UTIL}.")
    print(f"Kết quả được lưu tại: {OUTPUT_CSV_RESULT}")
    print(df_hui.head(10))
else:
    print(f"Không tìm thấy tập mục có giá trị cao vượt ngưỡng {MIN_UTIL}.")
    # Tạo file CSV rỗng
    pd.DataFrame(columns=["Itemset", "Total_Utility"]).to_csv(OUTPUT_CSV_RESULT, index=False)